In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

txn_train = pd.read_csv("transaction_train.csv")
txn_test = pd.read_csv("transaction_test.csv")
campaign = pd.read_csv("campaign_table.csv")
coupon = pd.read_csv("coupon_redempt.csv")
demo = pd.read_csv("demographic_new.csv")

print("txn_train:",txn_train.shape)
print("txn_test:",txn_test.shape)
print("campaign:",campaign.shape)
print("coupon:",coupon.shape)
print("demo:",demo.shape)

txn_train: (1783033, 13)
txn_test: (713927, 13)
campaign: (7208, 5)
coupon: (2318, 6)
demo: (2200, 8)


In [2]:
df_train = txn_train.copy()

In [3]:
df_train.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC,DEPARTMENT
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0,PRODUCE
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0,PRODUCE
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0,PRODUCE
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0,PRODUCE
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0,PRODUCE


In [4]:
def get_indicator(d,k):
    if d == k:
        return 1
    else:
        return 0

In [5]:
for i in df_train.DEPARTMENT.unique():
    df_train[i+str("_ind")] = df_train.apply(lambda x: get_indicator(x.DEPARTMENT, i), axis =1)

In [6]:
df_train.shape

(1783033, 56)

In [8]:
#df_train.to_csv("df_train.csv")

In [9]:
df_train.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,...,MEAT-WHSE_ind,HBC_ind,ELECT &PLUMBING_ind,HOUSEWARES_ind,PHARMACY SUPPLY_ind,GRO BAKERY_ind,TOYS_ind,VIDEO_ind,PORK_ind,CHARITABLE CONT_ind
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,...,0,0,0,0,0,0,0,0,0,0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,...,0,0,0,0,0,0,0,0,0,0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,...,0,0,0,0,0,0,0,0,0,0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,...,0,0,0,0,0,0,0,0,0,0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df1= df_train.loc[:, ["household_key","BASKET_ID","STORE_ID",  "DAY","WEEK_NO", "TRANS_TIME","DEPARTMENT","QUANTITY","SALES_VALUE", "RETAIL_DISC"]].groupby(["household_key","BASKET_ID","STORE_ID",  "DAY","WEEK_NO", "TRANS_TIME","DEPARTMENT"], as_index=False)["QUANTITY","SALES_VALUE", "RETAIL_DISC"].sum()
df1

<ipython-input-10-7118d264471e>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df1= df_train.loc[:, ["household_key","BASKET_ID","STORE_ID",  "DAY","WEEK_NO", "TRANS_TIME","DEPARTMENT","QUANTITY","SALES_VALUE", "RETAIL_DISC"]].groupby(["household_key","BASKET_ID","STORE_ID",  "DAY","WEEK_NO", "TRANS_TIME","DEPARTMENT"], as_index=False)["QUANTITY","SALES_VALUE", "RETAIL_DISC"].sum()


,household_key,BASKET_ID,STORE_ID,DAY,WEEK_NO,TRANS_TIME,DEPARTMENT,QUANTITY,SALES_VALUE,RETAIL_DISC
0,1,27601281299,436,51,8,1456,DELI,1,3.71,-0.62
1,1,27601281299,436,51,8,1456,DRUG GM,2,3.00,-0.58
2,1,27601281299,436,51,8,1456,GROCERY,23,50.22,-10.35
3,1,27601281299,436,51,8,1456,MEAT-PCKGD,5,12.98,-4.47
4,1,27601281299,436,51,8,1456,PASTRY,2,7.98,0.00
...,...,...,...,...,...,...,...,...,...,...
471485,2500,42250641157,447,708,102,1620,GROCERY,11,19.53,-9.60
471486,2500,42250641157,447,708,102,1620,MEAT,3,7.18,-2.97
471487,2500,42250641157,447,708,102,1620,MISC SALES TRAN,2,3.00,0.00
471488,2500,42250641157,447,708,102,1620,NUTRITION,1,1.50,-1.49


In [12]:
df_1a=df1.loc[:, ["BASKET_ID","DEPARTMENT","QUANTITY"]].pivot(index='BASKET_ID', columns='DEPARTMENT', values='QUANTITY').reset_index()
df_1a.fillna(0, inplace = True)
df_1a.columns = ['BASKET_ID', ' _qty', 'AUTOMOTIVE_qty', 'CHARITABLE_CONT_qty', 'CHEF_SHOPPE_qty',
       'CNTRL_STORE_SUP_qty', 'COSMETICS_qty', 'COUP/STR&MFG_qty', 'DAIRY_DELI_qty', 'DELI_qty',
       'DELI_SNACK_BAR_qty', 'DRUG_GM_qty', 'ELECT&PLUMBING_qty', 'FLORAL_qty',
       'FROZEN_GROCERY_qty', 'GARDEN_CENTER_qty', 'GM_MERCH_EXP_qty', 'GRO_BAKERY_qty',
       'GROCERY_qty', 'HBC_qty', 'HOUSEWARES_qty', 'KIOSK_GAS_qty', 'MEAT_qty', 'MEAT_PCKGD_qty',
       'MEAT-WHSE_qty', 'MISC_SALES TRAN_qty', 'MISC_TRANS_qty', 'NUTRITION_qty', 'PASTRY_qty',
       'PHARMACY SUPPLY_qty', 'PHOTO_qty', 'PORK_qty', 'POSTAL CENTER_qty', 'PRODUCE_qty',
       'RESTAURANT_qty', 'RX_qty', 'SALAD_BAR_qty', 'SEAFOOD_qty', 'SEAFOOD_PCKGD_qty', 'SPIRITS_qty',
       'TOYS_qty', 'TRAVEL&LEISUR_qty', 'VIDEO_qty', 'VIDEO_RENTAL_qty']
df_1a

,BASKET_ID,_qty,AUTOMOTIVE_qty,CHARITABLE_CONT_qty,CHEF_SHOPPE_qty,CNTRL_STORE_SUP_qty,COSMETICS_qty,COUP/STR&MFG_qty,DAIRY_DELI_qty,DELI_qty,...,RESTAURANT_qty,RX_qty,SALAD_BAR_qty,SEAFOOD_qty,SEAFOOD_PCKGD_qty,SPIRITS_qty,TOYS_qty,TRAVEL&LEISUR_qty,VIDEO_qty,VIDEO_RENTAL_qty
0,26984851472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,26984851516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,26984896261,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,26984905972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,26984945254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188837,42291245957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188838,42291245965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188839,42297590710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188840,42305362497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df_1b=df1.loc[:, ["BASKET_ID","DEPARTMENT","SALES_VALUE"]].pivot(index='BASKET_ID', columns='DEPARTMENT', values='SALES_VALUE').reset_index()
df_1b.fillna(0, inplace = True)
df_1b.columns = ['BASKET_ID', ' _sales', 'AUTOMOTIVE_sales', 'CHARITABLE_CONT_sales', 'CHEF_SHOPPE_sales',
       'CNTRL_STORE_SUP_sales', 'COSMETICS_sales', 'COUP/STR&MFG_sales', 'DAIRY_DELI_sales', 'DELI_sales',
       'DELI_SNACK_BAR_sales', 'DRUG_GM_sales', 'ELECT&PLUMBING_sales', 'FLORAL_sales',
       'FROZEN_GROCERY_sales', 'GARDEN_CENTER_sales', 'GM_MERCH_EXP_sales', 'GRO_BAKERY_sales',
       'GROCERY_sales', 'HBC_sales', 'HOUSEWARES_sales', 'KIOSK_GAS_sales', 'MEAT_sales', 'MEAT_PCKGD_sales',
       'MEAT-WHSE_sales', 'MISC_SALES TRAN_sales', 'MISC_TRANS_sales', 'NUTRITION_sales', 'PASTRY_sales',
       'PHARMACY SUPPLY_sales', 'PHOTO_sales', 'PORK_sales', 'POSTAL CENTER_sales', 'PRODUCE_sales',
       'RESTAURANT_sales', 'RX_sales', 'SALAD_BAR_sales', 'SEAFOOD_sales', 'SEAFOOD_PCKGD_sales', 'SPIRITS_sales',
       'TOYS_sales', 'TRAVEL&LEISUR_sales', 'VIDEO_sales', 'VIDEO_RENTAL_sales']
df_1b

,BASKET_ID,_sales,AUTOMOTIVE_sales,CHARITABLE_CONT_sales,CHEF_SHOPPE_sales,CNTRL_STORE_SUP_sales,COSMETICS_sales,COUP/STR&MFG_sales,DAIRY_DELI_sales,DELI_sales,...,RESTAURANT_sales,RX_sales,SALAD_BAR_sales,SEAFOOD_sales,SEAFOOD_PCKGD_sales,SPIRITS_sales,TOYS_sales,TRAVEL&LEISUR_sales,VIDEO_sales,VIDEO_RENTAL_sales
0,26984851472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,26984851516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,26984896261,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,26984905972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,26984945254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188837,42291245957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.49,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188838,42291245965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188839,42297590710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188840,42305362497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
df_1c=df1.loc[:, ["BASKET_ID","DEPARTMENT","RETAIL_DISC"]].pivot(index='BASKET_ID', columns='DEPARTMENT', values='RETAIL_DISC').reset_index()
df_1c.fillna(0, inplace = True)
df_1c.columns =['BASKET_ID', ' _retail_disc', 'AUTOMOTIVE_retail_disc', 'CHARITABLE_CONT_retail_disc', 'CHEF_SHOPPE_retail_disc',
       'CNTRL_STORE_SUP_retail_disc', 'COSMETICS_retail_disc', 'COUP/STR&MFG_retail_disc', 'DAIRY_DELI_retail_disc', 'DELI_retail_disc',
       'DELI_SNACK_BAR_retail_disc', 'DRUG_GM_retail_disc', 'ELECT&PLUMBING_retail_disc', 'FLORAL_retail_disc',
       'FROZEN_GROCERY_retail_disc', 'GARDEN_CENTER_retail_disc', 'GM_MERCH_EXP_retail_disc', 'GRO_BAKERY_retail_disc',
       'GROCERY_retail_disc', 'HBC_retail_disc', 'HOUSEWARES_retail_disc', 'KIOSK_GAS_retail_disc', 'MEAT_retail_disc', 'MEAT_PCKGD_retail_disc',
       'MEAT-WHSE_retail_disc', 'MISC_retail_disc TRAN_retail_disc', 'MISC_TRANS_retail_disc', 'NUTRITION_retail_disc', 'PASTRY_retail_disc',
       'PHARMACY SUPPLY_retail_disc', 'PHOTO_retail_disc', 'PORK_retail_disc', 'POSTAL CENTER_retail_disc', 'PRODUCE_retail_disc',
       'RESTAURANT_retail_disc', 'RX_retail_disc', 'SALAD_BAR_retail_disc', 'SEAFOOD_retail_disc', 'SEAFOOD_PCKGD_retail_disc', 'SPIRITS_retail_disc',
       'TOYS_retail_disc', 'TRAVEL&LEISUR_retail_disc', 'VIDEO_retail_disc', 'VIDEO_RENTAL_retail_disc']
df_1c

,BASKET_ID,_retail_disc,AUTOMOTIVE_retail_disc,CHARITABLE_CONT_retail_disc,CHEF_SHOPPE_retail_disc,CNTRL_STORE_SUP_retail_disc,COSMETICS_retail_disc,COUP/STR&MFG_retail_disc,DAIRY_DELI_retail_disc,DELI_retail_disc,...,RESTAURANT_retail_disc,RX_retail_disc,SALAD_BAR_retail_disc,SEAFOOD_retail_disc,SEAFOOD_PCKGD_retail_disc,SPIRITS_retail_disc,TOYS_retail_disc,TRAVEL&LEISUR_retail_disc,VIDEO_retail_disc,VIDEO_RENTAL_retail_disc
0,26984851472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,26984851516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,26984896261,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,26984905972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,26984945254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188837,42291245957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.49,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188838,42291245965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188839,42297590710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188840,42305362497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
df_ = pd.merge(df_1a, df_1b, on = "BASKET_ID", how = "inner")
df_dep_agg_hh = pd.merge(df_, df_1c, on = "BASKET_ID", how = "inner")
df_dep_agg_hh

,BASKET_ID,_qty,AUTOMOTIVE_qty,CHARITABLE_CONT_qty,CHEF_SHOPPE_qty,CNTRL_STORE_SUP_qty,COSMETICS_qty,COUP/STR&MFG_qty,DAIRY_DELI_qty,DELI_qty,...,RESTAURANT_retail_disc,RX_retail_disc,SALAD_BAR_retail_disc,SEAFOOD_retail_disc,SEAFOOD_PCKGD_retail_disc,SPIRITS_retail_disc,TOYS_retail_disc,TRAVEL&LEISUR_retail_disc,VIDEO_retail_disc,VIDEO_RENTAL_retail_disc
0,26984851472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,26984851516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,26984896261,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,26984905972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,26984945254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188837,42291245957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188838,42291245965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188839,42297590710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188840,42305362497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
df2= df_train.drop(["PRODUCT_ID", "DEPARTMENT"], axis = 1).groupby(["household_key","BASKET_ID","STORE_ID",  "DAY","WEEK_NO", "TRANS_TIME"], as_index=False).sum()
df2

,household_key,BASKET_ID,STORE_ID,DAY,WEEK_NO,TRANS_TIME,QUANTITY,SALES_VALUE,RETAIL_DISC,COUPON_DISC,...,MEAT-WHSE_ind,HBC_ind,ELECT &PLUMBING_ind,HOUSEWARES_ind,PHARMACY SUPPLY_ind,GRO BAKERY_ind,TOYS_ind,VIDEO_ind,PORK_ind,CHARITABLE CONT_ind
0,1,27601281299,436,51,8,1456,34,78.66,-16.54,-1.0,...,0,0,0,0,0,0,0,0,0,0
1,1,27774192959,436,67,10,2002,14,41.10,-8.59,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1,28024266849,436,88,13,1604,13,26.90,-6.72,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1,28106322445,436,94,14,1937,32,63.43,-11.08,-1.5,...,0,0,0,0,0,0,0,0,0,0
4,1,28235481967,436,101,15,1603,20,53.45,-16.42,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188837,2500,42076591137,447,695,100,1618,2,4.00,-0.98,0.0,...,0,0,0,0,0,0,0,0,0,0
188838,2500,42115225458,327,698,100,1136,12,50.54,-14.72,-2.0,...,0,0,0,0,0,0,0,0,0,0
188839,2500,42198526427,447,704,101,1531,31,73.65,-19.79,-1.7,...,0,0,0,0,0,0,0,0,0,0
188840,2500,42229520973,447,706,102,1527,6,13.95,-0.42,0.0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
aggregated_data = pd.merge(df2, df_dep_agg_hh, on = "BASKET_ID", how = "inner")
aggregated_data.shape

(188842, 183)

In [18]:
aggregated_data.head()

,household_key,BASKET_ID,STORE_ID,DAY,WEEK_NO,TRANS_TIME,QUANTITY,SALES_VALUE,RETAIL_DISC,COUPON_DISC,...,RESTAURANT_retail_disc,RX_retail_disc,SALAD_BAR_retail_disc,SEAFOOD_retail_disc,SEAFOOD_PCKGD_retail_disc,SPIRITS_retail_disc,TOYS_retail_disc,TRAVEL&LEISUR_retail_disc,VIDEO_retail_disc,VIDEO_RENTAL_retail_disc
0,1,27601281299,436,51,8,1456,34,78.66,-16.54,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,27774192959,436,67,10,2002,14,41.10,-8.59,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,28024266849,436,88,13,1604,13,26.90,-6.72,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,28106322445,436,94,14,1937,32,63.43,-11.08,-1.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,28235481967,436,101,15,1603,20,53.45,-16.42,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
### adding key variables

In [20]:
df3 = df_train.groupby(["household_key","BASKET_ID","STORE_ID"], as_index = False)["PRODUCT_ID"].nunique()
df3.columns = ["household_key","BASKET_ID","STORE_ID", "count_diff_items_bought"] #sum of qty : total items bought
df3

,household_key,BASKET_ID,STORE_ID,count_diff_items_bought
0,1,27601281299,436,30
1,1,27774192959,436,12
2,1,28024266849,436,12
3,1,28106322445,436,23
4,1,28235481967,436,17
...,...,...,...,...
188837,2500,42076591137,447,2
188838,2500,42115225458,327,12
188839,2500,42198526427,447,27
188840,2500,42229520973,447,6


In [24]:
aggregated_data = aggregated_data.merge(df3)
aggregated_data.shape

(188842, 184)

In [25]:
df4 = df_train.groupby(["household_key","BASKET_ID","STORE_ID"], as_index = False)["DEPARTMENT"].nunique()
df4.columns = ["household_key","BASKET_ID","STORE_ID", "no_of_depts_shopped"]
df4

,household_key,BASKET_ID,STORE_ID,no_of_depts_shopped
0,1,27601281299,436,6
1,1,27774192959,436,3
2,1,28024266849,436,4
3,1,28106322445,436,4
4,1,28235481967,436,5
...,...,...,...,...
188837,2500,42076591137,447,1
188838,2500,42115225458,327,2
188839,2500,42198526427,447,4
188840,2500,42229520973,447,2


In [26]:
aggregated_data = aggregated_data.merge(df4)
aggregated_data.shape

(188842, 185)

In [32]:
#import pickle
#pickle.dump(aggregated_data, open('Desktop', 'wb'))

In [36]:
hh1 = aggregated_data[aggregated_data.household_key ==1]
hh1.to_csv("hh1.csv")

# aggregating at HH level

In [39]:
711/30, 711-90

(23.7, 621)

In [40]:
aggregated_data_21m = aggregated_data[aggregated_data.DAY < 621]
aggregated_data_21m.shape

(163692, 185)

### creating new vars at HH level
- recency
- tenure
- avg gap between nutr orders

In [59]:
temp = aggregated_data_21m.loc[:, ["household_key","BASKET_ID", "NUTRITION_ind","DAY"]]
temp1 = temp[temp.NUTRITION_ind > 0]
temp1 = temp1.groupby(["household_key"], as_index = False)["DAY"].last()
temp1.columns = ["household_key", "last_day_nutr"]
temp1["recency_nutr"] = 620 - temp1["last_day_nutr"]
df5 = temp1.loc[:, ["household_key", "recency_nutr"]]
df5

,household_key,recency_nutr
0,1,39
1,4,266
2,5,31
3,7,10
4,9,516
...,...,...
1126,2494,195
1127,2495,176
1128,2496,249
1129,2498,32


In [62]:
temp = aggregated_data_21m.loc[:, ["household_key","BASKET_ID","NUTRITION_ind","DAY"]]
temp1 = temp[temp.NUTRITION_ind > 0]
temp1["lagged"] = temp1.groupby(["household_key"])['DAY'].shift(1)
temp1["diff"] = temp1["DAY"] - temp1["lagged"]
tmp1 = temp1.dropna(subset = ["diff"])
df6 = tmp1.groupby(["household_key"], as_index = False)["diff"].first()
df6.columns = ["household_key", "days_to_second_order_nut"]
df6

<ipython-input-62-41026cddb578>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp1["lagged"] = temp1.groupby(["household_key"])['DAY'].shift(1)
<ipython-input-62-41026cddb578>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp1["diff"] = temp1["DAY"] - temp1["lagged"]


,household_key,days_to_second_order_nut
0,1,39.0
1,5,2.0
2,7,5.0
3,13,109.0
4,15,253.0
...,...,...
831,2489,39.0
832,2490,38.0
833,2496,23.0
834,2498,70.0


In [63]:
temp2 = temp1.loc[:, ["household_key", "diff"]]
df7 = temp2.groupby(["household_key"], as_index = False).mean()
df7.columns = ["household_key", "avg_gap_in_orders_nut"]
df7

,household_key,avg_gap_in_orders_nut
0,1,18.333333
1,4,NaN
2,5,2.000000
3,7,29.500000
4,9,NaN
...,...,...
1126,2494,NaN
1127,2495,NaN
1128,2496,23.000000
1129,2498,47.428571


In [69]:
min_max_days= aggregated_data_21m.loc[:, ["household_key","DAY"]].groupby(["household_key"], as_index = False).agg(["min", "max"])
min_max_days = min_max_days.reset_index()
min_max_days.columns = ["household_key", "min_day", "max_day"]
min_max_days["tenure"] = min_max_days["max_day"]-min_max_days["min_day"]
min_max_df = min_max_days.loc[:, ["household_key", "tenure"]]

In [77]:
drop_cols = ["BASKET_ID", "STORE_ID", "DAY", "WEEK_NO", "TRANS_TIME"]
aggregated_data_21m_1 = aggregated_data_21m.drop(drop_cols,axis =1)
aggregated_data_21m_2 = aggregated_data_21m_1.groupby(["household_key"], as_index = False).sum()
aggregated_data_21m_2.shape

(1749, 180)

In [81]:
df_merge1 = pd.merge(aggregated_data_21m_2, df5, on = "household_key", how = "left")
df_merge2 = pd.merge(df_merge1, df6, on = "household_key", how = "left")
df_merge3 = pd.merge(df_merge2, df7, on = "household_key", how = "left")
df_merge4 = pd.merge(df_merge3, min_max_df, on = "household_key", how = "left")
df_merge4.shape

(1749, 184)

In [82]:
df_merge4.head()

,household_key,QUANTITY,SALES_VALUE,RETAIL_DISC,COUPON_DISC,COUPON_MATCH_DISC,PRODUCE_ind,GROCERY_ind,DRUG GM_ind,SEAFOOD-PCKGD_ind,...,TOYS_retail_disc,TRAVEL&LEISUR_retail_disc,VIDEO_retail_disc,VIDEO_RENTAL_retail_disc,count_diff_items_bought,no_of_depts_shopped,recency_nutr,days_to_second_order_nut,avg_gap_in_orders_nut,tenure
0,1,1725,3693.77,-621.64,-73.27,-21.25,126,990,139,0,...,0.0,0.0,0.0,0.0,1482,293,39.0,39.0,18.333333,569
1,3,8472,2554.98,-650.88,-61.19,-19.80,36,653,68,2,...,0.0,0.0,0.0,0.0,870,138,NaN,NaN,NaN,463
2,4,374,1175.16,-114.28,-2.50,0.00,3,225,30,3,...,0.0,0.0,0.0,0.0,295,75,266.0,NaN,NaN,513
3,5,242,749.09,-118.33,0.00,0.00,2,120,51,0,...,0.0,0.0,0.0,0.0,219,83,31.0,2.0,2.000000,504
4,7,1081,2420.12,-331.19,-16.00,-2.25,74,548,165,0,...,0.0,0.0,0.0,0.0,897,164,10.0,5.0,29.500000,591


## merging demo data 

In [103]:
demo

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,15-24K,Homeowner,Unknown,1,None/Unknown,3
2,19-24,B,50-74K,Homeowner,1 Adult Kids,2,None/Unknown,4
3,25-34,A,35-49K,Unknown,2 Adults Kids,3,None/Unknown,5
4,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
...,...,...,...,...,...,...,...,...
2195,45-54,A,75-99K,Homeowner,Unknown,3,1,2496
2196,45-54,U,35-49K,Unknown,Single Male,1,None/Unknown,2497
2197,25-34,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2498
2198,25-34,U,Under 15K,Unknown,2 Adults Kids,3,1,2499


In [105]:
one_hot_encoded_data = pd.get_dummies(demo, columns =['AGE_DESC', 'MARITAL_STATUS_CODE', 'INCOME_DESC', 'HOMEOWNER_DESC',
       'HH_COMP_DESC', 'HOUSEHOLD_SIZE_DESC', 'KID_CATEGORY_DESC'])
one_hot_encoded_data

,household_key,AGE_DESC_19-24,AGE_DESC_25-34,AGE_DESC_35-44,AGE_DESC_45-54,AGE_DESC_55-64,AGE_DESC_65+,MARITAL_STATUS_CODE_A,MARITAL_STATUS_CODE_B,MARITAL_STATUS_CODE_U,...,HH_COMP_DESC_Unknown,HOUSEHOLD_SIZE_DESC_1,HOUSEHOLD_SIZE_DESC_2,HOUSEHOLD_SIZE_DESC_3,HOUSEHOLD_SIZE_DESC_4,HOUSEHOLD_SIZE_DESC_5+,KID_CATEGORY_DESC_1,KID_CATEGORY_DESC_2,KID_CATEGORY_DESC_3+,KID_CATEGORY_DESC_None/Unknown
0,1,0,0,0,0,0,1,1,0,0,...,0,0,1,0,0,0,0,0,0,1
1,3,0,0,0,1,0,0,1,0,0,...,1,1,0,0,0,0,0,0,0,1
2,4,1,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
3,5,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
4,7,0,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,2496,0,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,1,0,0,0
2196,2497,0,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
2197,2498,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
2198,2499,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,1,0,0,0


In [106]:
df_merge4_demo = pd.merge(df_merge4, one_hot_encoded_data, on = "household_key", how = "left")
df_merge4_demo.shape

(1749, 225)

## merging coupon

In [129]:
coupon_nut = coupon[coupon.Department == "NUTRITION"]
coupon_nut["coupon_nut_red"] = 1
coupon_nut_df = coupon_nut.loc[:, ["household_key","coupon_nut_red" ]]

<ipython-input-129-a2bb3a5f8a2e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coupon_nut["coupon_nut_red"] = 1


In [132]:
df_merge4_demo_wc = pd.merge(df_merge4_demo, coupon_nut_df, on = "household_key", how = "left")
df_merge4_demo_wc.shape

(1750, 226)

In [55]:
#creating a target variable

In [41]:
aggregated_data_3m = aggregated_data[aggregated_data.DAY >= 621]
aggregated_data_3m.shape

(25150, 185)

In [42]:
get_target = aggregated_data_3m.loc[:, ["household_key","BASKET_ID", "NUTRITION_ind"]]
get_target.head()

,household_key,BASKET_ID,NUTRITION_ind
73,1,41008476635,0
74,1,41238565364,1
75,1,41338185846,0
76,1,41439966015,0
77,1,41453231209,0


In [56]:
get_target1= get_target.groupby(["household_key"], as_index = False)["NUTRITION_ind"].sum()

In [96]:
get_target1["Target"] = get_target1["NUTRITION_ind"].apply(lambda x: 1 if x > 0 else 0)
get_target1

,household_key,NUTRITION_ind,Target
0,1,2,1
1,3,0,0
2,4,0,0
3,5,0,0
4,7,4,1
...,...,...,...
1607,2495,0,0
1608,2496,0,0
1609,2497,2,1
1610,2498,4,1


In [97]:
y_train = get_target1.loc[:,["household_key","Target"]]
y_train.shape

(1612, 2)

In [83]:
y_train.Target.value_counts()

0    1042
1     570
Name: Target, dtype: int64

### merging with DV

In [133]:
X_train = df_merge4_demo_wc.copy()
X_train.shape

(1750, 226)

In [134]:
train_finaldf = pd.merge(df_merge4_demo_wc, y_train, on = "household_key", how = "left")
train_finaldf.shape

(1750, 227)

In [135]:
train_finaldf.Target.value_counts()

0.0    1041
1.0     571
Name: Target, dtype: int64

In [136]:
y_train.head(2)

,household_key,Target
0,1,1
1,3,0


In [137]:
train_finaldf.head()

,household_key,QUANTITY,SALES_VALUE,RETAIL_DISC,COUPON_DISC,COUPON_MATCH_DISC,PRODUCE_ind,GROCERY_ind,DRUG GM_ind,SEAFOOD-PCKGD_ind,...,HOUSEHOLD_SIZE_DESC_2,HOUSEHOLD_SIZE_DESC_3,HOUSEHOLD_SIZE_DESC_4,HOUSEHOLD_SIZE_DESC_5+,KID_CATEGORY_DESC_1,KID_CATEGORY_DESC_2,KID_CATEGORY_DESC_3+,KID_CATEGORY_DESC_None/Unknown,coupon_nut_red,Target
0,1,1725,3693.77,-621.64,-73.27,-21.25,126,990,139,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0
1,3,8472,2554.98,-650.88,-61.19,-19.80,36,653,68,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,0.0
2,4,374,1175.16,-114.28,-2.50,0.00,3,225,30,3,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,0.0
3,5,242,749.09,-118.33,0.00,0.00,2,120,51,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,0.0
4,7,1081,2420.12,-331.19,-16.00,-2.25,74,548,165,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0


In [138]:
#train_finaldf.to_csv("train_df_21m.csv") #no demo and coupon
#train_finaldf.to_csv("train_df_21m_v2.csv") #with demo and coupon nutrition indicator